# Query Apache Iceberg Tables with DuckDB Native Extension + JupySQL

This notebook demonstrates querying Iceberg tables using:
- **DuckDB Iceberg Extension** - Native Iceberg support (works with Minio, not R2)
- **SQLite Catalog** - Local file-based catalog for metadata
- **JupySQL** - SQL magic commands for Jupyter notebooks

## Prerequisites
```bash
pip install duckdb jupysql duckdb-engine pandas
```

## Requirements
- Minio running on `localhost:9000`
- SQLite catalog file (`iceberg_catalog.db`) with table metadata

In [ ]:
# Install required packages
%pip install duckdb jupysql duckdb-engine pandas -q

## 1. Configuration

In [13]:
# Minio Configuration
S3_ENDPOINT = 'localhost:9000'
S3_ACCESS_KEY = 'minioadmin'
S3_SECRET_KEY = 'minioadmin'
S3_REGION = 'us-east-1'
S3_USE_SSL = 'false'

# SQLite Catalog
CATALOG_DB = './iceberg_catalog.db'

# Table to query
NAMESPACE = 'ticketdb'
TABLE_NAME = 'netflix_titles'

print(f"Minio Endpoint: {S3_ENDPOINT}")
print(f"Catalog DB: {CATALOG_DB}")
print(f"Table: {NAMESPACE}.{TABLE_NAME}")

Minio Endpoint: localhost:9000
Catalog DB: ./iceberg_catalog.db
Table: ticketdb.netflix_titles


## 2. Get Metadata Location from SQLite Catalog

The SQLite catalog stores the current metadata file location for each table.
This avoids hardcoding UUIDs like `00001-baeb619d-03ff-44fe-b0c0-efabd32cf55a.metadata.json`.

In [14]:
import sqlite3

def get_metadata_location(catalog_db: str, namespace: str, table_name: str) -> str:
    """Query SQLite catalog to get the current metadata location for a table."""
    conn = sqlite3.connect(catalog_db)
    cursor = conn.execute(
        "SELECT metadata_location FROM iceberg_tables WHERE table_namespace = ? AND table_name = ?",
        (namespace, table_name)
    )
    result = cursor.fetchone()
    conn.close()
    
    if result:
        return result[0]
    else:
        raise ValueError(f"Table {namespace}.{table_name} not found in catalog")

# Get metadata location
METADATA_LOCATION = get_metadata_location(CATALOG_DB, NAMESPACE, TABLE_NAME)
print(f"📍 Metadata Location: {METADATA_LOCATION}")

📍 Metadata Location: s3://iceberg/warehouse/netflix_titles/metadata/00001-baeb619d-03ff-44fe-b0c0-efabd32cf55a.metadata.json


## 3. List All Tables in Catalog

In [15]:
import sqlite3
import pandas as pd

def list_all_tables(catalog_db: str) -> pd.DataFrame:
    """List all tables in the SQLite catalog."""
    conn = sqlite3.connect(catalog_db)
    df = pd.read_sql_query(
        "SELECT table_namespace, table_name, metadata_location FROM iceberg_tables",
        conn
    )
    conn.close()
    return df

# Show all tables
tables_df = list_all_tables(CATALOG_DB)
print("📁 Tables in Catalog:")
tables_df

📁 Tables in Catalog:


,table_namespace,table_name,metadata_location
0,ticketdb,netflix_titles,s3://iceberg/warehouse/netflix_titles/metadata...


## 6. Setup JupySQL

In [18]:
# Load JupySQL extension
%load_ext sql

# Configure JupySQL
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = True
%config SqlMagic.displaycon = False

# Connect JupySQL using duckdb-engine
# We'll reconfigure S3 settings after connection
%sql duckdb:///:memory:

# Get the underlying DuckDB connection and configure it
from sql import connection
raw_conn = connection.ConnectionManager.current._connection.connection.dbapi_connection

# Install and load extensions on this connection
raw_conn.execute("INSTALL iceberg; LOAD iceberg;")
raw_conn.execute("INSTALL httpfs; LOAD httpfs;")

# Configure S3/Minio settings
raw_conn.execute(f"SET s3_endpoint='{S3_ENDPOINT}';")
raw_conn.execute(f"SET s3_access_key_id='{S3_ACCESS_KEY}';")
raw_conn.execute(f"SET s3_secret_access_key='{S3_SECRET_KEY}';")
raw_conn.execute(f"SET s3_use_ssl={S3_USE_SSL};")
raw_conn.execute("SET s3_url_style='path';")
raw_conn.execute(f"SET s3_region='{S3_REGION}';")

# Create the view on this connection
raw_conn.execute(f"""
    CREATE OR REPLACE VIEW {TABLE_NAME} AS 
    SELECT * FROM iceberg_scan('{METADATA_LOCATION}')
""")

# Verify
result = raw_conn.execute(f"SELECT COUNT(*) FROM {TABLE_NAME}").fetchone()
print(f"✅ JupySQL connected with Iceberg extension")
print(f"✅ View '{TABLE_NAME}' created with {result[0]:,} rows")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
✅ JupySQL connected with Iceberg extension
✅ View 'netflix_titles' created with 49,990 rows


## 7. Query with SQL Magic ✨

Now we can write pure SQL using `%%sql` magic!

In [19]:
%%sql
-- Count total rows
SELECT COUNT(*) as total_users FROM netflix_titles

,total_users
0,49990


In [20]:
%%sql
-- View table schema
DESCRIBE netflix_titles

,Success


In [21]:
%%sql
-- Sample rows
SELECT userid, username, firstname, lastname, city, state 
FROM netflix_titles 
LIMIT 10

,userid,username,firstname,lastname,city,state
0,1,JSG99FHE,Rafael,Taylor,Kent,WA
1,2,PGL08LJI,Vladimir,Humphrey,Murfreesboro,SK
2,3,IFT66TXU,Lars,Ratliff,High Point,ME
3,4,XDZ38RDD,Barry,Roy,Omaha,AB
4,5,AEB55QTM,Reagan,Hodge,Forest Lake,NS
5,6,NDQ15VBM,Victor,Hernandez,Naperville,GA
6,7,OWY35QYB,Tamekah,Juarez,Moultrie,WV
7,8,AZG78YIP,Colton,Roy,Guayama,AK
8,9,MSD36KVR,Mufutau,Watkins,Port Orford,MD
9,10,WKW41AIW,Naida,Calderon,Waterbury,MB


In [22]:
%%sql
-- Users by State (Top 10)
SELECT 
    state,
    COUNT(*) as user_count
FROM netflix_titles
GROUP BY state
ORDER BY user_count DESC
LIMIT 10

,state,user_count
0,NT,1998
1,NB,1960
2,BC,1958
3,QC,1929
4,YT,1919
5,NL,1919
6,MB,1916
7,PE,1906
8,NS,1896
9,AB,1894


In [24]:
%%sql
-- User preferences analysis
SELECT 
    'Sports' as preference, COUNT(*) as count FROM netflix_titles WHERE likesports = true
UNION ALL
SELECT 'Theatre', COUNT(*) FROM netflix_titles WHERE liketheatre = true
UNION ALL
SELECT 'Concerts', COUNT(*) FROM netflix_titles WHERE likeconcerts = true
UNION ALL
SELECT 'Jazz', COUNT(*) FROM netflix_titles WHERE likejazz = true
UNION ALL
SELECT 'Classical', COUNT(*) FROM netflix_titles WHERE likeclassical = true
UNION ALL
SELECT 'Opera', COUNT(*) FROM netflix_titles WHERE likeopera = true
UNION ALL
SELECT 'Rock', COUNT(*) FROM netflix_titles WHERE likerock = true
UNION ALL
SELECT 'Vegas', COUNT(*) FROM netflix_titles WHERE likevegas = true
UNION ALL
SELECT 'Broadway', COUNT(*) FROM netflix_titles WHERE likebroadway = true
UNION ALL
SELECT 'Musicals', COUNT(*) FROM netflix_titles WHERE likemusicals = true
ORDER BY count DESC

,preference,count
0,Musicals,12607
1,Theatre,12534
2,Classical,12523
3,Concerts,12512
4,Jazz,12441
5,Opera,12429
6,Vegas,12414
7,Broadway,12389
8,Rock,12380
9,Sports,8682


In [25]:
%%sql
-- Find users who like both Sports and Jazz
SELECT 
    userid,
    username,
    firstname,
    lastname,
    city,
    state
FROM netflix_titles
WHERE likesports = true AND likejazz = true
LIMIT 10

,userid,username,firstname,lastname,city,state
0,33,PFD07GEF,Shafira,Glenn,Starkville,WA
1,67,TWU10MZT,Herman,Myers,Basin,PE
2,87,DUF19VXU,Reese,Merrill,Marietta,NC
3,114,BPO22XTW,Preston,Harding,Meadville,ON
4,121,AQI41NWD,Aaron,Smith,Malden,QC
5,130,ZQC82ALK,Nayda,Mcgee,Santa Rosa,HI
6,135,RKV66ZIW,Teegan,Caldwell,Laramie,NB
7,141,RRB46OBP,Howard,Day,Honolulu,NT
8,143,ABS88QAU,Raphael,Leon,Roseville,PA
9,184,TVX65AZX,Hermione,Jacobs,Sierra Madre,YT


## 8. Save Query Results to Variable

In [26]:
%%sql result <<
SELECT state, COUNT(*) as count 
FROM netflix_titles 
GROUP BY state 
ORDER BY count DESC

In [27]:
# Result is already a DataFrame (autopandas=True)
df = result
df.head(10)

,state,count
0,NT,1998
1,NB,1960
2,BC,1958
3,QC,1929
4,YT,1919
5,NL,1919
6,MB,1916
7,PE,1906
8,NS,1896
9,AB,1894


## 10. Helper: Query Any Table Dynamically

In [ ]:
def query_iceberg_table(catalog_db: str, namespace: str, table_name: str, sql_query: str):
    """
    Dynamically query any Iceberg table by looking up metadata from SQLite catalog.
    
    Args:
        catalog_db: Path to SQLite catalog database
        namespace: Iceberg namespace
        table_name: Iceberg table name
        sql_query: SQL query (use 'table' as placeholder for table name)
    
    Returns:
        pandas DataFrame with query results
    """
    import duckdb
    import sqlite3
    
    # Get metadata location from catalog
    cat_conn = sqlite3.connect(catalog_db)
    cursor = cat_conn.execute(
        "SELECT metadata_location FROM iceberg_tables WHERE table_namespace = ? AND table_name = ?",
        (namespace, table_name)
    )
    result = cursor.fetchone()
    cat_conn.close()
    
    if not result:
        raise ValueError(f"Table {namespace}.{table_name} not found")
    
    metadata_location = result[0]
    
    # Setup DuckDB
    conn = duckdb.connect()
    conn.execute("INSTALL iceberg; LOAD iceberg;")
    conn.execute("INSTALL httpfs; LOAD httpfs;")
    conn.execute(f"SET s3_endpoint='{S3_ENDPOINT}';")
    conn.execute(f"SET s3_access_key_id='{S3_ACCESS_KEY}';")
    conn.execute(f"SET s3_secret_access_key='{S3_SECRET_KEY}';")
    conn.execute(f"SET s3_use_ssl={S3_USE_SSL};")
    conn.execute("SET s3_url_style='path';")
    conn.execute(f"SET s3_region='{S3_REGION}';")
    
    # Create view and run query
    conn.execute(f"CREATE VIEW tbl AS SELECT * FROM iceberg_scan('{metadata_location}')")
    
    # Replace 'table' placeholder with 'tbl'
    final_query = sql_query.replace('{table}', 'tbl')
    
    return conn.execute(final_query).fetchdf()

# Example usage
result = query_iceberg_table(
    CATALOG_DB,
    'ticketdb',
    'netflix_titles',
    "SELECT state, COUNT(*) as cnt FROM {table} GROUP BY state ORDER BY cnt DESC LIMIT 5"
)
result

---

## What We Built

This notebook demonstrates querying Iceberg tables using **DuckDB's native Iceberg extension** with local Minio storage.

**Key difference from the PyIceberg approach:**
- DuckDB reads Iceberg files directly from S3/Minio
- No Arrow table bridge needed
- Works with Minio (local) but **not** with Cloudflare R2 (SSL issues)

**The SQLite catalog lookup** solves the "how do I find the metadata file" problem — no need to remember UUIDs like `00001-baeb619d-03ff-44fe-b0c0-efabd32cf55a.metadata.json`.

### Tech Stack
- **Storage**: Minio (localhost:9000)
- **Catalog**: SQLite (iceberg_catalog.db)
- **Query Engine**: DuckDB with Iceberg extension
- **Interface**: JupySQL

**Cost: $0** (all local/open source)